In [ ]:
import logging
from pathlib import Path
from sandbox.qemu import QemuVMManager, VMConfig


source = Path("./docker/shared") / "test"
print(source)
# # Set up a logger for the test
logger = logging.getLogger("TestQemuVMManager")
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)


# Set up a test VM configuration.
config = VMConfig(
    image="qemux/qemu",  # Use your actual QEMU image
    container_name="test_qemu",  # A test container name
    base_vm_path="./docker/vms/ubuntu-noble",  # Temporary base directory created above
    shared_path="./docker/shared",  # Temporary shared storage directory
)

# # Instantiate the QemuVMManager.
manager = QemuVMManager(config=config, logger=logger)

manager.get_or_create_container()


# manager.cleanup_vm()


2025-04-17 09:56:21,348 - TestQemuVMManager - INFO - VM state changed: created -> created


docker/shared/test


2025-04-17 09:56:54,647 - TestQemuVMManager - INFO - Creating container test_qemu with storage at /home/smwhering/IS-master-thesis/src/docker/environments/test_qemu_1744876581
2025-04-17 09:56:54,649 - TestQemuVMManager - INFO - Binding host docker/shared/test_qemu_1744876581 → container /shared (tag=test_qemu_1744876581)
2025-04-17 09:56:54,649 - TestQemuVMManager - INFO - Checking image qemux/qemu locally...
2025-04-17 09:56:54,658 - TestQemuVMManager - INFO - Image qemux/qemu exists.
2025-04-17 09:56:54,698 - TestQemuVMManager - INFO - Container test_qemu created
2025-04-17 09:56:54,699 - TestQemuVMManager - INFO - VM state changed: created -> starting
2025-04-17 09:56:54,984 - TestQemuVMManager - INFO - Container test_qemu started
2025-04-17 09:56:54,985 - TestQemuVMManager - INFO - VM state changed: starting -> running
2025-04-17 09:56:54,986 - TestQemuVMManager - INFO - SSH connection attempt 1/5...
2025-04-17 09:56:54,988 - TestQemuVMManager - INFO - Connection attempt failed: [

<Container: ea0f0e403612>